In [2]:
"""

1. llmsearch on gsm8k awq with batch size 1
2. rerun on a bigger search space

"""

# Autocompletion
%config Completer.use_jedi = False

# Autoreload
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/workspace/llmsearch')

import gc
import torch
import ctypes
import json
import nltk
import math
import torch
import random
import evaluate
import datasets
import langchain
import numpy as np
import pandas as pd
import transformers
from transformers import GPTQConfig, BitsAndBytesConfig
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import PreTrainedModel, PretrainedConfig, GenerationConfig, StoppingCriteria, AutoTokenizer, StoppingCriteriaList, AutoModel, AutoModelForCausalLM

import os
import gc
import ctypes
import traceback
from pathlib import Path
from typing import Any, Dict, Optional, Union, List

import time
import textwrap
from tqdm.auto import tqdm

from exllamav2 import (
    ExLlamaV2,
    ExLlamaV2Cache,
    ExLlamaV2Cache_8bit,
    ExLlamaV2Config
)

from datasets import load_dataset
from llmsearch.model_downloader import download_model_from_hf
from llmsearch.utils.model_utils import batcher, decoder_parser

import awq

from awq import AutoAWQForCausalLM

def pretty_print_dict(d, indent = 4):
    print(json.dumps(d, indent = indent, default = str))

Monkey Patching .generate function of `transformers` library


In [3]:
gsm8k_dataset = load_dataset("gsm8k", 'main')

torch.__version__, awq.__version__

('2.2.0+cu121', '0.2.4')

In [4]:
def seed_everything(seed):
    """Seed for reproducibilty"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def cm():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

# loaders

def load_model_with_awq_backend(model_id, model_loader_kwargs, tokenizer_kwargs,temp_model_dir, model_branch = "main"):
    output_folder = download_model_from_hf(model_id, save_dir = temp_model_dir, branch = model_branch)

    model_loader_kwargs['pretrained_model_name_or_path'] = output_folder
    tokenizer_loader_kwargs['pretrained_model_name_or_path'] = output_folder

    model_name_or_path = model_loader_kwargs.pop('pretrained_model_name_or_path')
    model = AutoAWQForCausalLM.from_quantized(
        quant_path=model_name_or_path,
        **model_loader_kwargs
    )
    tokenizer = AutoTokenizer.from_pretrained(**tokenizer_kwargs, local_files_only=True)

    # pad token is null in config -https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-AWQ/blob/eb64c310c44905321d012962db9ac0d47c3a64fa/tokenizer_config.json#L53
    tokenizer.pad_token = tokenizer.unk_token

    return model, tokenizer

model_loader_backend_map = {
    # "exllama_2_hf": load_model_with_exllama_2_hf_backend,
    # "hf": load_model_with_hf_backend,
    # 'auto_gptq' : load_model_with_autogptq_backend,
    'awq' : load_model_with_awq_backend,
}

In [5]:
# https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-AWQ
model_id = "TheBloke/CapybaraHermes-2.5-Mistral-7B-AWQ"

temp_model_dir = Path(f"/workspace/temp_model_dir/")
temp_model_dir.mkdir(exist_ok = True, parents = True)

model_loader_kwargs = {
    'device_map' : {'' : 0},
    'fuse_layers' : True,
}

tokenizer_loader_kwargs = {
    'use_fast' : False,
    'legacy' : False,
    'padding_side' : 'left',
}

model, tokenizer = load_model_with_awq_backend(model_id, model_loader_kwargs, tokenizer_loader_kwargs,temp_model_dir, model_branch = "main")

Model already exists in /workspace/temp_model_dir/TheBloke_CapybaraHermes-2.5-Mistral-7B-AWQ. Checking the model files...
Checksum validated: model.safetensors  645dfc7f09074aaf25e642f3c6a4f7ea399a0ff2605fa650e4e74078832546de
Checksum validated: tokenizer.model  dadfd56d766715c61d2ef780a525ab43b8e6da4de6865bda3d95fdef5e134055
[+] Validated checksums of all model files!


Fusing layers...: 100%|██████████| 32/32 [00:02<00:00, 12.85it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
pt = textwrap.dedent("""\
    Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
    A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

    Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
    A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

    Q: {question}""")
pt_cols = ['question']
system_prompt = "Solve the following math problems, end with The answer is"

# Add prompt template
processed_dataset = preprocess_dataset(gsm8k_dataset['train'], tokenizer,pt = pt, pt_cols = pt_cols, system_prompt = system_prompt, add_generation_prompt = True)

In [8]:
import re

def extract_answer_from_out(s):
    pattern = re.compile(r"The answer is (\d+(?:\.\d+)?)")
    match = pattern.search(s)
    if match:
        return match.group(1).strip()
    else:
        return None

def get_score(y_true, y_pred):
    scores = []

    for y_t, y_p in zip(y_true, y_pred):
        y_t_answer = y_t['answer'].split("####")[-1].strip()
        y_p_answer = extract_answer_from_out(y_p)


        if y_t_answer == y_p_answer:
            scores.append(1)
        else:
            scores.append(0)
    return sum(scores)/len(scores)

In [9]:
from llmsearch.utils.logging_utils import set_verbosity_info, set_verbosity_debug, set_verbosity_warning
set_verbosity_warning()

In [10]:
seed = 42
bm_sample_size = 150
bm_samples = processed_dataset.shuffle(seed = seed).select(range(bm_sample_size))

In [11]:

from llmsearch.tuner import Tuner

# TODO : bs = 16 error, figure this out - done - issue in stopping criteria script
# TODO : run llmsearch - done

class MultiTokenEOSCriteria2(transformers.StoppingCriteria):
    """Criteria to stop on the specified multi-token sequence.

    This code is not thread safe. The same object cannot be used simultaneously in multiple threads.
    """

    def __init__(
        self,
        sequence_ids : List[int],
    ) -> None:
        self.sequence_ids = torch.tensor(sequence_ids, dtype = torch.int32, device = "cuda:0")
        # we look back for 2 more tokens than it takes to encode our stop sequence
        # because tokenizers suck, and a model might generate `['\n', '\n']` but our `sequence` is `['\n\n']`
        # and we don't want to mistakenly not stop a generation because our
        # (string) stop sequence was output in a different tokenization
        # NOTE: there is a minor danger that this will end up looking back 2 tokens into the past, into the inputs to the model,
        # and stopping generation immediately as a result. With only 2 extra tokens of lookback, this risk is minimized
        # Additionally, in lookback_ids_batch we should prevent ever looking back into the inputs as described.
        self.sequence_id_len = self.sequence_ids.shape[0] + 2
        self.state_initialized = False
        self.prompt_length = None
        self.state_initialized = False

    def set_state(self, batch_size, prompt_length):
        self.batch_size = batch_size
        self.prompt_length = prompt_length
        self.done_tracker = [False] * batch_size
        self.state_initialized = True

    def reset(self):
        # print("Resetting")
        self.batch_size = None
        self.prompt_length = None
        self.state_initialized = False


    def __call__(self, input_ids, scores, **kwargs) -> bool:
        """
        This is called after a new token is generated
        """
        # For efficiency, we compare the last n tokens where n is the number of tokens in the stop_sequence

        ret_val = False

        if not self.state_initialized:
            # Every batch should set this state
            # print(f"Setting state, batch_size - {input_ids.shape[0]}, batch prompt length - {input_ids.shape[1] - 1}")
            self.set_state(input_ids.shape[0], input_ids.shape[1] - 1)

        # IDs of all the tokens except the prompt
        lookback_ids_batch = input_ids[:, self.prompt_length :]
        # look back for 2 more tokens than it takes to encode our stop sequence
        lookback_ids_batch = lookback_ids_batch[:, -self.sequence_id_len :]

        # print(f"Current input length - {input_ids.shape[1]}, completion length - {abs(self.prompt_length - input_ids.shape[1])}")
        # print(f"Current input - {tokenizer.batch_decode(input_ids, **{'spaces_between_special_tokens' : False})}")

        # no elements yet to look back
        if lookback_ids_batch.nelement() == 0:
            return False

        for i, done in enumerate(self.done_tracker):
            if not done:
                # look back only as far as the last token of the stop sequence
                # print(len(self.done_tracker), lookback_ids_batch.shape, self.batch_size, self.prompt_length)
                self.done_tracker[i] = self.sequence_ids == lookback_ids_batch[i][-(self.sequence_ids.shape[0]):]
        ret_val = False not in self.done_tracker
        if ret_val:
            # ASSUMPTION: Relies on the assumption that generation will only stop when the stop token is generated
            self.reset()
        return ret_val

In [12]:
multi_token_stop_criteria_ob = MultiTokenEOSCriteria2(sequence_ids = [32000])
stopping_criteria = StoppingCriteriaList([multi_token_stop_criteria_ob])

batch_size = 2
tuner_ob = Tuner(
    model = model,
    tokenizer = tokenizer,
    dataset = bm_samples,
    device = 'cuda:0',
    batch_size = batch_size,
    tokenizer_encoding_kwargs={'padding': 'longest', 'add_special_tokens' : False},
    tokenizer_decoding_kwargs={'spaces_between_special_tokens' : False},
    scorer = get_score,
    prompt_template = langchain.PromptTemplate.from_template("{X}"),
    is_encoder_decoder = False,
    seed = seed,
    column_mapping = {'input_cols' : ["X"],'eval_cols' : ['answer']},
    callbacks_after_inference = [multi_token_stop_criteria_ob.reset],
)



In [13]:
gen_params1 = {
    'max_new_tokens' : 500,
    # max_new_tokens take precendece over stopping criteria
    'stopping_criteria' : stopping_criteria,
    'generation_seed' : 42,
}

scores_before, outputs_before = tuner_ob.get_score(gen_params1)

  0%|          | 0/75 [00:00<?, ?it/s]

In [14]:
print(scores_before)

0.6533333333333333


In [15]:
from sklearn.metrics import make_scorer

hyp_space = {
    'max_new_tokens' : [500],
    'stopping_criteria' : [stopping_criteria],
    'generation_seed' : [42],
    'do_sample' : [True],

    'temperature': [0.8,0.9,1.0],  # Continuous distribution from 0.1 to 1.0
    'top_k': [10,50,60,70,80],  # Discrete uniform distribution from 1 to 100
    'top_p' : [0.7,0.75,0.8,0.95],
    'no_repeat_ngram_size': [0],

}


scorer = make_scorer(score_func=get_score, greater_is_better=True)

In [16]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
clf = GridSearchCV(
    estimator = tuner_ob.estimator,
    param_grid=hyp_space,
    scoring = scorer,
    cv = 2,
    n_jobs = None,
    verbose=3,
)

In [17]:
clf.fit(X=tuner_ob.dataset["X"], y=tuner_ob.dataset['y'])

Fitting 2 folds for each of 60 candidates, totalling 120 fits


  0%|          | 0/38 [00:00<?, ?it/s]

[CV 1/2] END do_sample=True, generation_seed=42, max_new_tokens=500, no_repeat_ngram_size=0, stopping_criteria=[<__main__.MultiTokenEOSCriteria2 object at 0x7f9fb00fd930>], temperature=0.8, top_k=10, top_p=0.7;, score=0.667 total time= 2.6min


  0%|          | 0/38 [00:00<?, ?it/s]

[CV 2/2] END do_sample=True, generation_seed=42, max_new_tokens=500, no_repeat_ngram_size=0, stopping_criteria=[<__main__.MultiTokenEOSCriteria2 object at 0x7f9fb00fd930>], temperature=0.8, top_k=10, top_p=0.7;, score=0.640 total time= 2.3min


...

  0%|          | 0/38 [00:00<?, ?it/s]

[CV 1/2] END do_sample=True, generation_seed=42, max_new_tokens=500, no_repeat_ngram_size=0, stopping_criteria=[<__main__.MultiTokenEOSCriteria2 object at 0x7f9fb00fd930>], temperature=1.0, top_k=80, top_p=0.95;, score=0.680 total time= 2.6min


  0%|          | 0/38 [00:00<?, ?it/s]

[CV 2/2] END do_sample=True, generation_seed=42, max_new_tokens=500, no_repeat_ngram_size=0, stopping_criteria=[<__main__.MultiTokenEOSCriteria2 object at 0x7f9fb00fd930>], temperature=1.0, top_k=80, top_p=0.95;, score=0.653 total time= 2.3min


GridSearchCV(cv=2,
             estimator=LLMEstimatorWrapper(batch_size=2,
                                           callbacks_after_inference=[<bound method MultiTokenEOSCriteria2.reset of <__main__.MultiTokenEOSCriteria2 object at 0x7f9fb00fd930>>],
                                           device='cuda:0',
                                           disable_batch_size_cache=False,
                                           do_sample=True, generation_seed=42,
                                           is_encoder_decoder=False,
                                           is_fitted_=True, max_new_tokens=500,
                                           model=MistralAWQForCa...
                                           top_k=50, top_p=0.95),
             param_grid={'do_sample': [True], 'generation_seed': [42],
                         'max_new_tokens': [500], 'no_repeat_ngram_size': [0],
                         'stopping_criteria': [[<__main__.MultiTokenEOSCriteria2 object at 0x7f9fb00fd930>]],
                         'temperature': [0.8, 0.9, 1.0],
                         'top_k': [10, 50, 60, 70, 80],
                         'top_p': [0.7, 0.75, 0.8, 0.95]},
             scoring=make_scorer(get_score, response_method='predict'),
             verbose=3)

In [18]:
scores_after, outputs_after = tuner_ob.get_score(clf.best_params_)

  0%|          | 0/75 [00:00<?, ?it/s]

In [19]:

print(scores_before, scores_after)

0.6533333333333333 0.6733333333333333


In [20]:
len(bm_samples) * scores_before, len(bm_samples) * scores_after

(98.0, 101.0)

In [21]:
len(bm_samples)

150

In [22]:
# TODO : evaluate on a bigger sample, currently evaluated on 150 size sample
str(clf.best_params_)

"{'do_sample': True, 'generation_seed': 42, 'max_new_tokens': 500, 'no_repeat_ngram_size': 0, 'stopping_criteria': [<__main__.MultiTokenEOSCriteria2 object at 0x7f9fb00fd930>], 'temperature': 0.9, 'top_k': 50, 'top_p': 0.95}"

In [23]:
import json

def json_dump(ob : dict, file_path: Path):
    with open(file_path, 'w', encoding="utf-8") as json_file:
        json.dump(ob, json_file, indent=4)

d = {
    'scores_before' : scores_before,
    'scores_after' : scores_after,
    'outputs_before' : outputs_before,
    'outputs_after' : outputs_after,
    'best_params' : str(clf.best_params_),
}

f = "./results-2-gsm8k-capybara-bs-1.json"
json_dump(d, f)